# Evaluate different policies in the simulated environment

## Description of the simulated environment
1. Goal: Detect terminal event with the smallest cost
1. Label: {0, 1}, 0 indicates normal, 5 1's indicates terminal event
1. Reponse: a time series composed of 0 and 1
    1. Generate under a markov chain
        1. $p(L_{t+1} = 0 |L_{t} = 0) = 0.9$
        1. $p(L_{t+1} = 0 |L_{t} = 1) = 0.1$
        1. $p(L_{t+1} = 1 |L_{t} = 0) = 1.0$
        1. $p(L_{t+1} = 1 |L_{t} = 1) = 0.0$
    1. The series terminate with 5 1's
1. Time series measurements generated according to the label
    1. 3 informative features
        1. $1 \pm \epsilon$ if $L=1$
        1. $-1 \pm \epsilon$ if $L=0$
    1. 3 noisy features
    1. Randomly introduced missingness to create incompleted dataset
        1. missing value is encoder as $0$
1. Size of Training + Val set: 5000

    
## Description of classifier for event forcasting 
1. A softmax that takes in
    1. a state consisiting of measurements of the 5 most recent time steps and,
    1. a feature importance vector
        1. first three features have increasing importance
        1. last three features have some closs to zero importance
1. A Softmax output a probability of event occur at or after 5 time steps
    

## Description of the RL problem
1. State: a state consisting of measurements of the 5 most recent time steps
1. Actions: all possible measurement that can be done for the next time point
    1. (6 measurements) x 2
    1. For each measurement, the agent may choose to do it or not do it 
1. Rewards
    1. A 6-dimensional vector, the reward of each measurement is assumped to be independent of reward of another measurement
    1. Difference in cross entropy loss between an action vs no any measurement performed.


## Evaluation in an online setting
1. Agent observ state $s_t$
1. Agent decide action $a_t$
1. Environment generate next state $s_{t+1}$


In [1]:
import argparse
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import tensorflow as tf

from arch.K_Tails_Dueling_DQN import KTailsDuelingDQN
from simulated_patient_database import DummyDisease, DummyClassifier, SimulatedPatientDatabaseDiscrete
from sim_observation_dummy_disease import ExperienceGeneratorMixed

In [2]:
def get_random_actions(s, action_dim):
    batch_size = len(s)
    return np.random.randint(0, 2, (batch_size, action_dim))


def get_random_informative_action(s, action_dim, useful_action_dim):
    batch_size = len(s)
    return np.concatenate((np.random.randint(0, 2, (batch_size, useful_action_dim)),
                           np.zeros((batch_size, action_dim - useful_action_dim))), axis=1)


def get_all_actions(s, action_dim):
    batch_size = len(s)
    return np.ones((batch_size, action_dim))


def get_all_informative_actions(s, action_dim, useful_action_dim):
    batch_size = len(s)
    return np.concatenate((np.ones((batch_size, useful_action_dim)),
                           np.zeros((batch_size, action_dim - useful_action_dim))), axis=1)

def get_single_informative_actions(s, action_dim, indices):
    batch_size = len(s)
    a = np.zeros((batch_size, action_dim))
    a[:, indices] = 1
    return a

In [3]:
threshold = 0.65

In [ ]:
def parse_args():
    parser = argparse.ArgumentParser(description='Evaluate policy')

    parser.add_argument('--seed', type=int, default=0)
    parser.add_argument('--reward_params', type=float, default=(105,))

    parser.add_argument('--num_patients', type=int, default=500)

    parser.add_argument('--early_detect_time', type=int, default=5)

    parser.add_argument('--dataset_setting_path', type=str,
                        default='../data/simulated_disease/1015_hyperparameters.pkl')
    parser.add_argument('--data_keep_rate', type=float, default=0.9)

    args = parser.parse_args([])

    args.dqn_model_path_big_g = '../models/ktdqn-dummy1017-o30-a6-g0.99-m10000-nn1-2-64-lr1e-05-1e-05-0.5-' \
                                's64-5000-i20000-500-50-r(105,)-d0.9-r0/'
    return args

In [ ]:
args = parse_args()

np.random.seed(args.seed)
tf.set_random_seed(args.seed)

dataset_setting = pickle.load(open(args.dataset_setting_path, 'rb'))

classifier = DummyClassifier(num_useful_features=dataset_setting.num_useful_features,
                             num_noisy_features=dataset_setting.num_noisy_features,
                             obs_t_dim=dataset_setting.max_num_terminal_states,
                             score_param_1=dataset_setting.score_param_1)

disease = DummyDisease(feature_noise=dataset_setting.feature_noise,
                       max_num_terminal_states=dataset_setting.max_num_terminal_states,
                       num_useful_features=dataset_setting.num_useful_features,
                       num_noisy_features=dataset_setting.num_noisy_features,
                       num_datapoints_per_period=dataset_setting.num_datapoints_per_period,
                       period_length=dataset_setting.period_length,
                       min_periods=dataset_setting.min_periods, max_periods=dataset_setting.max_periods,
                       keep_rate=args.data_keep_rate)

patient_database = SimulatedPatientDatabaseDiscrete(obs_type=disease, num_patients=args.num_patients)

print('Rate of death: {}/{}'.format(np.sum(patient_database.all_labels), args.num_patients))

exp_generator = \
    ExperienceGeneratorMixed(patient_database=patient_database,
                             classifier=classifier,
                             early_detect_time=args.early_detect_time)

def reward_func(cur_action, information_gain):
    a = args.reward_params

    information_gain = a * information_gain
    action_cost = - cur_action
    return information_gain + action_cost


def evaluate_dqn_policy(dqn_model_path, get_action_random, threshold, e_greedy=1.0):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    dqn_hyperparam = pickle.load(open(dqn_model_path + 'hyperparameters.pkl', 'rb'))
    dqn_results = pickle.load(open(dqn_model_path + 'results.pkl', 'rb'))

    tf.reset_default_graph()

    with tf.Session(config=config) as sess:
        with tf.variable_scope('dqn', reuse=tf.AUTO_REUSE):
            dqn = \
                KTailsDuelingDQN(variable_scope='dqn', state_dim=dqn_hyperparam.state_dim,
                                 action_dim=dqn_hyperparam.action_dim,
                                 gamma=dqn_hyperparam.gamma, lr=dqn_hyperparam.lr,
                                 keep_prob=dqn_hyperparam.keep_prob,
                                 reg_constant=dqn_hyperparam.reg_constant,
                                 num_shared_all_layers=dqn_hyperparam.num_shared_all_layers,
                                 num_shared_dueling_layers=dqn_hyperparam.num_shared_dueling_layers,
                                 num_hidden_units=dqn_hyperparam.num_hidden_units,
                                 replace_target_batch=dqn_hyperparam.replace_target_batch,
                                 memory_size=dqn_hyperparam.memory_size, log=None)

        saver = tf.train.Saver()
        saver.restore(sess, dqn_model_path + 'model-%d' % dqn_results['best_model_idx'])

        get_action = lambda s: dqn.get_best_actions(sess=sess, s=s) if np.random.rand() <= e_greedy \
            else get_action_random(s=s)

        cur_state, actions = exp_generator.evaluate_with_deterministic_statte(get_action=get_action)

        print(actions)

        avg_reward, avg_action, accuracy = exp_generator.evaluate(get_action=get_action, reward_func=reward_func,
                                                                  threshold=threshold)

        # print('Average reward: {}, Avg action: {}'.format(avg_reward, avg_action))
        return avg_reward, avg_action, accuracy


def evaluate_easy_policy(get_action, threshold):
    avg_reward, avg_action, accuracy = exp_generator.evaluate(get_action=get_action, reward_func=reward_func,
                                                              threshold=threshold)

    # print('Average reward: {}, Avg action: {}'.format(avg_reward, avg_action))
    return avg_reward, avg_action, accuracy



Rate of death: 351/500


In [ ]:
get_action_random = lambda s: get_random_informative_action(s, action_dim=disease.num_features,
                                                            useful_action_dim=disease.num_useful_features)
get_action_all = lambda s: get_all_informative_actions(s, action_dim=disease.num_features,
                                                      useful_action_dim=disease.num_useful_features)
get_action_single0 = lambda s: get_single_informative_actions(s, action_dim=disease.num_features, indices=[0])
get_action_single1 = lambda s: get_single_informative_actions(s, action_dim=disease.num_features, indices=[1])
get_action_single2 = lambda s: get_single_informative_actions(s, action_dim=disease.num_features, indices=[2])
get_action_single12 = lambda s: get_single_informative_actions(s, action_dim=disease.num_features, indices=[1, 2])
get_action_single02 = lambda s: get_single_informative_actions(s, action_dim=disease.num_features, indices=[0, 2])

In [ ]:
resules = [evaluate_easy_policy(get_action=get_action_random, threshold=threshold), 
           evaluate_easy_policy(get_action=get_action_all, threshold=threshold),
           evaluate_easy_policy(get_action=get_action_single0, threshold=threshold),
           evaluate_easy_policy(get_action=get_action_single1, threshold=threshold),
           evaluate_easy_policy(get_action=get_action_single2, threshold=threshold),
           evaluate_easy_policy(get_action=get_action_single12, threshold=threshold),
          evaluate_easy_policy(get_action=get_action_single02, threshold=threshold)]

finished: 0/500
finished: 10/500
finished: 20/500
finished: 30/500


/h/maimingj/projects/autodiagnosis/simulated_database/sim_observation_dummy_disease.py:374: RuntimeWarning: divide by zero encountered in true_divide
  ratio = information_gain_true / information_gain_sum
/h/maimingj/projects/autodiagnosis/simulated_database/sim_observation_dummy_disease.py:374: RuntimeWarning: invalid value encountered in true_divide
  ratio = information_gain_true / information_gain_sum


finished: 40/500
finished: 50/500
finished: 60/500
finished: 70/500
finished: 80/500
finished: 90/500
finished: 100/500
finished: 110/500
finished: 120/500
finished: 130/500
finished: 140/500
finished: 150/500
finished: 160/500
finished: 170/500
finished: 180/500
finished: 190/500
finished: 200/500
finished: 210/500
finished: 220/500
finished: 230/500
finished: 240/500
finished: 250/500
finished: 260/500
finished: 270/500
finished: 280/500
finished: 290/500
finished: 300/500
finished: 310/500
finished: 320/500
finished: 330/500
finished: 340/500
finished: 350/500
finished: 360/500
finished: 370/500
finished: 380/500
finished: 390/500
finished: 400/500
finished: 410/500
finished: 420/500
finished: 430/500
finished: 440/500
finished: 450/500
finished: 460/500
finished: 470/500
finished: 480/500
finished: 490/500
finished: 0/500
finished: 10/500
finished: 20/500
finished: 30/500
finished: 40/500
finished: 50/500
finished: 60/500
finished: 70/500
finished: 80/500
finished: 90/500
finished:

In [ ]:
for i, (r, a, acc) in enumerate(resules):
    print('\n Result: {}'.format(i))
    print('rew mean', np.mean(np.sum(r, axis=1), axis=0))
    print('rew std',  np.std(np.sum(r, axis=1), axis=0))
    print('act freq', np.mean(a))
    print('accuracy', np.mean(acc))


 Result: 0
rew mean -47.11242196928637
rew std 52.449573838453354
act freq 0.2486314768782416
accuracy 0.376

 Result: 1
rew mean -84.74966932511042
rew std 86.19073000693241
act freq 0.5
accuracy 0.968

 Result: 2
rew mean -20.047109724753007
rew std 28.526034222166786
act freq 0.16666666666666663
accuracy 0.298

 Result: 3
rew mean -20.59467829610527
rew std 32.793269509738
act freq 0.16666666666666663
accuracy 0.298

 Result: 4
rew mean -21.124063289162738
rew std 37.28632422412663
act freq 0.16666666666666663
accuracy 0.298

 Result: 5
rew mean -48.503695470879954
rew std 60.96289658319845
act freq 0.33333333333333326
accuracy 0.938

 Result: 6
rew mean -50.00745213809427
rew std 58.570423366589026
act freq 0.33333333333333326
accuracy 0.662


In [ ]:
res = evaluate_dqn_policy(dqn_model_path=args.dqn_model_path_big_g, get_action_random=get_action_random, threshold=threshold)

INFO:tensorflow:Restoring parameters from ../models/ktdqn-dummy1017-o30-a6-g0.99-m10000-nn1-2-64-lr1e-05-1e-05-0.5-s64-5000-i20000-500-50-r(105,)-d0.9-r0/model-42
[[0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 1 0 0 0]
 [0 1 1 0 0 0]
 [0 1 1 0 0 0]
 [0 1 1 0 0 0]]
finished: 0/500
finished: 10/500
finished: 20/500
finished: 30/500
finished: 40/500
finished: 50/500
finished: 60/500
finished: 70/500
finished: 80/500
finished: 90/500
finished: 100/500
finished: 110/500
finished: 120/500
finished: 130/500


In [ ]:
r, a, acc = res
print('rew mean', np.mean(np.sum(r, axis=1), axis=0))
print('rew std',  np.std(np.sum(r, axis=1), axis=0))
print('act freq', np.mean(a))
print('acc', np.mean(acc))